<a href="https://colab.research.google.com/github/sajidcsecu/radioGenomic/blob/main/3DUnetinGPU_(Nifti_MONAI2Balanced).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is the Code for the Segmentation on Rider Dataset (LUNG1). The Code is worked on the 3D volume over GPU. The balanced sampler, preprocessed data (uniform volume spacing and clipping [-1000, 700]) and the strong augmentation is used in the code...

# (1) Import Required Libraries

In [1]:
!pip install SimpleITK
!pip install pydicom===2.4.3
!pip install pydicom-seg
!pip install numpy==1.23.5
!pip install monai
!pip install torch==1.13.1
!pip install nibabel>=5.0.0

  Using cached numpy-1.23.5.tar.gz (10.7 MB)
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 38.3 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement torch==1.13.1 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.7.1, 2.8.0)
ERROR: No matching distribution found for torch==1.13.1



# (2) Import required Libraries

In [2]:
import os
from pathlib import Path
import csv
import time
import random
import shutil
from glob import glob
from typing import List
import torch
import numpy as np
import torch.nn as nn
import torch.cuda.amp as amp
from torch.optim import lr_scheduler
from monai.inferers import sliding_window_inference
from monai.transforms import AsDiscrete
from monai.networks.nets import UNet
from monai.transforms import (
    Compose,
    LoadImaged,
    EnsureChannelFirstd,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
    ResizeWithPadOrCropd,
    RandCropByPosNegLabeld,
    RandFlipd,
    RandAffined,
    RandGaussianNoised,
    RandScaleIntensityd,
    ToTensord,
    EnsureTyped,
    EnsureChannelFirstD,
    SpatialPadd
)
from monai.data import Dataset, DataLoader, CacheDataset, pad_list_data_collate
from monai.networks.layers import Norm
import nibabel as nib
from sklearn.metrics import jaccard_score, f1_score, recall_score, precision_score, accuracy_score
import matplotlib.pyplot as plt
import multiprocessing as mp
from monai.transforms import EnsureTyped
from monai.transforms import SaveImaged
from monai.utils import set_determinism
import warnings
warnings.filterwarnings("ignore", message="Num foregrounds 0, Num backgrounds.*unable to generate class balanced samples")

# (3) Mount Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## (4). Loss Function

In [4]:
# ---------- Combined Dice + BCE loss for 3D segmentation ----------
class DiceBCELoss3D(nn.Module):
    def __init__(self, smooth=1e-6, epsilon=1e-8, dice_w=1.0, bce_w=1.0):
        super().__init__()
        self.smooth = smooth
        self.epsilon = epsilon
        self.dice_w = dice_w
        self.bce_w = bce_w
        self.bce = nn.BCEWithLogitsLoss()

    def set_pos_weight(self, pos_weight=None, device="cpu"):
        if pos_weight is None:
            self.bce = nn.BCEWithLogitsLoss()
        else:
            tensor_pw = torch.tensor([pos_weight], dtype=torch.float, device=device)
            self.bce = nn.BCEWithLogitsLoss(pos_weight=tensor_pw)
        return self

    def forward(self, preds, targets):
        preds_sigmoid = torch.sigmoid(preds)

        inter = (preds_sigmoid * targets).sum()
        dice_loss = 1 - (2. * inter + self.smooth) / (
            preds_sigmoid.sum() + targets.sum() + self.smooth + self.epsilon
        )

        bce_loss = self.bce(preds, targets)
        return self.dice_w * dice_loss + self.bce_w * bce_loss

# (5). Test

In [5]:
class UnetTest:
    def __init__(self, test_result_path: str, metrics_csv: str, device: torch.device):
        self.test_result_path = test_result_path
        self.metrics_csv = metrics_csv
        self.device = device

        os.makedirs(self.test_result_path, exist_ok=True)
        self._init_metrics_csv()

    def _init_metrics_csv(self):
        if not os.path.exists(self.metrics_csv):
            with open(self.metrics_csv, 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(["SampleID", "Jaccard", "F1", "Recall", "Precision", "Accuracy", "Time"])

    def calculate_metrics(self, y_true: np.ndarray, y_pred: np.ndarray):
        y_true = y_true.astype(bool).flatten()
        y_pred = y_pred.astype(bool).flatten()

        return [
            jaccard_score(y_true, y_pred, zero_division=0),
            f1_score(y_true, y_pred, zero_division=0),
            recall_score(y_true, y_pred, zero_division=0),
            precision_score(y_true, y_pred, zero_division=0),
            accuracy_score(y_true, y_pred)
        ]

    def save_result_slices(self, image: np.ndarray, pred_mask: np.ndarray, true_mask: np.ndarray, sample_id: str):
        sample_dir = os.path.join(self.test_result_path, sample_id)
        os.makedirs(sample_dir, exist_ok=True)

        for i in range(image.shape[0]):
            try:
                fig, ax = plt.subplots(1, 3, figsize=(12, 4))
                ax[0].imshow(image[i], cmap='gray')
                ax[0].set_title('Image')

                ax[1].imshow(true_mask[i], cmap='gray')
                ax[1].set_title('Ground Truth')

                ax[2].imshow(pred_mask[i], cmap='gray')
                ax[2].set_title('Prediction')

                for a in ax:
                    a.axis('off')
                plt.tight_layout()
                plt.savefig(os.path.join(sample_dir, f'slice_{i:03d}.png'))
                plt.close()
            except Exception as e:
                print(f"⚠️ Could not save slice {i} for {sample_id}: {e}")

    def append_metrics_to_csv(self, sample_id: str, metrics: list, elapsed_time: float):
        with open(self.metrics_csv, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([sample_id] + [f"{m:.4f}" for m in metrics] + [f"{elapsed_time:.4f}"])

    def test(self, model: nn.Module, test_loader: DataLoader):
        model.eval()
        total_metrics = np.zeros(5)
        total_times = []

        roi_size = (96, 96, 96)
        sw_batch_size = 1

        with torch.no_grad():
            for batch_idx, batch in enumerate(test_loader):
                image, label = batch["vol"].to(self.device), batch["seg"].to(self.device)
                start_time = time.time()

                pred = sliding_window_inference(
                    inputs=image,
                    roi_size=roi_size,
                    sw_batch_size=sw_batch_size,
                    predictor=model
                )
                pred = torch.sigmoid(pred) > 0.5  # Binary thresholding

                elapsed = time.time() - start_time
                total_times.append(elapsed)

                # Convert to NumPy
                image_np = image[0, 0].cpu().numpy()
                label_np = label[0, 0].cpu().numpy()
                pred_np = pred[0, 0].cpu().numpy()

                # Metrics
                metrics = self.calculate_metrics(label_np, pred_np)
                total_metrics += np.array(metrics)

                # Identify sample name
                sample_id = os.path.basename(batch["vol_meta_dict"]["filename_or_obj"][0]).replace(".nii.gz", "")
                self.save_result_slices(image_np, pred_np, label_np, sample_id)
                self.append_metrics_to_csv(sample_id, metrics, elapsed)

        # Print summary
        num_samples = len(test_loader)
        print("\n📊 Average Test Metrics:")
        print(f"Jaccard:  {total_metrics[0]/num_samples:.4f}")
        print(f"F1:       {total_metrics[1]/num_samples:.4f}")
        print(f"Recall:   {total_metrics[2]/num_samples:.4f}")
        print(f"Precision:{total_metrics[3]/num_samples:.4f}")
        print(f"Accuracy: {total_metrics[4]/num_samples:.4f}")
        print(f"⚡ FPS:    {1 / np.mean(total_times):.2f}")

## (6) Early Stopping

In [6]:
class EarlyStopping:
    def __init__(self, patience=10, verbose=True, min_delta=0, path='checkpoint.pt',
                 start_val_loss_min=None, start_patience_counter=0):
        self.patience = patience
        self.verbose = verbose
        self.min_delta = min_delta
        self.path = path
        self.val_loss_min = start_val_loss_min if start_val_loss_min is not None else np.inf
        self.counter = start_patience_counter
        self.early_stop = False

    def __call__(self, val_loss, model, epoch=None, optimizer=None):
        improved = False
        if val_loss < self.val_loss_min - self.min_delta:
            self.val_loss_min = val_loss
            self.counter = 0
            improved = True
            if self.verbose:
                print(f"✅ Validation loss improved. Saving model...")
        else:
            self.counter += 1
            if self.verbose:
                print(f"⏳ EarlyStopping counter: {self.counter} out of {self.patience}")

        # Always save a full checkpoint
        self.save_checkpoint(model, epoch, optimizer)

        if self.counter >= self.patience:
            self.early_stop = True

        return self.early_stop

    def save_checkpoint(self, model, epoch=None, optimizer=None):
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict() if optimizer else None,
            'val_loss': self.val_loss_min,
            'patience_counter': self.counter
        }
        torch.save(checkpoint, self.path)

# (7). Training

In [7]:

# ---------- Trainer ----------
class UnetTrain:
    def __init__(self, model_file, loss_result_path, lr, num_epochs, device):
        self.model_file = model_file
        self.loss_result_path = loss_result_path
        self.lr = lr
        self.num_epochs = num_epochs
        self.device = device
        self.seeding(42)

    def seeding(self, seed):
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

    def epoch_time(self, start_time, end_time):
        elapsed = end_time - start_time
        return int(elapsed / 60), int(elapsed % 60)

    def compute_pos_weight(self, train_loader, max_batches=None):
        """Compute pos_weight = (#neg / #pos) from dataset."""
        pos, neg = 0, 0
        for i, batch in enumerate(train_loader):
            seg = batch["seg"].to(self.device)
            pos += seg.sum().item()
            neg += (seg.numel() - seg.sum().item())
            if max_batches and (i + 1) >= max_batches:
                break
        pos_weight = neg / (pos + 1e-8)
        print(f"📊 Auto-computed pos_weight = {pos_weight:.3f} "
              f"(pos voxels={pos:.0f}, neg voxels={neg:.0f})")
        return pos_weight

    def train_one_epoch(self, model, loader, optimizer, loss_fn):
        model.train()
        epoch_loss = 0
        scaler = torch.amp.GradScaler()
        device_type = 'cuda' if self.device.type == 'cuda' else 'cpu'

        for x in loader:
            inputs, labels = x["vol"].to(self.device), x["seg"].to(self.device)
            optimizer.zero_grad()
            with torch.amp.autocast(device_type=device_type):
                outputs = model(inputs)
                loss = loss_fn(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            epoch_loss += loss.item()
        return epoch_loss / len(loader)

    def evaluate(self, model, loader, loss_fn):
        model.eval()
        epoch_loss = 0
        with torch.no_grad():
            for x in loader:
                inputs, labels = x["vol"].to(self.device), x["seg"].to(self.device)
                outputs = model(inputs)
                loss = loss_fn(outputs, labels)
                epoch_loss += loss.item()
        return epoch_loss / len(loader)

    def execute(self, train_loader, valid_loader):
        model = UNet(
            spatial_dims=3,
            in_channels=1,
            out_channels=1,
            channels=(16, 32, 64, 128, 256),
            strides=(2, 2, 2, 2),
            num_res_units=2,
            norm=Norm.BATCH
        ).to(self.device)

        optimizer = torch.optim.AdamW(model.parameters(), lr=self.lr, weight_decay=1e-5)
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)

        # 🔹 Auto-compute pos_weight before training
        pos_weight = self.compute_pos_weight(train_loader, max_batches=50)
        loss_fn = DiceBCELoss3D().set_pos_weight(pos_weight, device=self.device)

        # ---- Resume training state ----
        start_epoch = 1
        start_val_loss_min = None
        start_patience_counter = 0
        history = {"train_loss": [], "valid_loss": []}

        if os.path.exists(self.model_file):
            checkpoint = torch.load(self.model_file, map_location=self.device)
            model.load_state_dict(checkpoint['model_state_dict'])
            if checkpoint.get('optimizer_state_dict'):
                optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            start_epoch = checkpoint.get('epoch', 1) + 1
            start_val_loss_min = checkpoint.get('val_loss', None)
            start_patience_counter = checkpoint.get('patience_counter', 0)

        if os.path.exists(self.loss_result_path):
            with open(self.loss_result_path, 'r') as f:
                reader = csv.reader(f)
                next(reader)
                rows = list(reader)
                if rows:
                    last_epoch = int(rows[-1][0])
                    start_epoch = last_epoch + 1
                    history['train_loss'] = [float(r[1]) for r in rows]
                    history['valid_loss'] = [float(r[2]) for r in rows]
                    if start_val_loss_min is None:
                        start_val_loss_min = min(history['valid_loss'])
            shutil.copy(self.loss_result_path, self.loss_result_path.replace(".csv", "_backup.csv"))

        early_stopping = EarlyStopping(
            patience=10,
            min_delta=0.0005,
            path=self.model_file,
            start_val_loss_min=start_val_loss_min,
            start_patience_counter=start_patience_counter
        )

        if not os.path.exists(self.loss_result_path):
            with open(self.loss_result_path, "w", newline="") as f:
                csv.writer(f).writerow(["Epoch", "Train Loss", "Valid Loss"])

        # ---- Training loop ----
        for epoch in range(start_epoch, self.num_epochs + 1):
            start_time = time.time()
            train_loss = self.train_one_epoch(model, train_loader, optimizer, loss_fn)
            valid_loss = self.evaluate(model, valid_loader, loss_fn)
            scheduler.step()

            mins, secs = self.epoch_time(start_time, time.time())
            print(f"Epoch {epoch:03d} | Time: {mins}m {secs}s | "
                  f"Train: {train_loss:.6f} | Val: {valid_loss:.6f}")

            history['train_loss'].append(train_loss)
            history['valid_loss'].append(valid_loss)

            with open(self.loss_result_path, "a", newline="") as f:
                csv.writer(f).writerow([epoch, train_loss, valid_loss])

            if early_stopping(valid_loss, model, epoch, optimizer):
                print("🛑 Early stopping triggered.")
                break

            torch.cuda.empty_cache()

# (8). Pipeline

In [ ]:
class UnetPipeline:
    def __init__(self, config):
        self.config = config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        set_determinism(seed=42)

        # paths
        os.chdir(self.config['target_dir'])
        print(f"📁 Current Directory: {os.getcwd()}")
        self.output_dir = os.path.join(".", "results", self.config['output_folder_name'])
        os.makedirs(self.output_dir, exist_ok=True)
        self.loss_result_file = os.path.join(self.output_dir, "train_and_valid_loss_results.csv")
        self.model_file = os.path.join(self.output_dir, "model.pth")
        self.test_metrics_file = os.path.join(self.output_dir, "test_metrics.csv")
        self.test_result_path = os.path.join(self.output_dir, "test_outputs")
        os.makedirs(self.test_result_path, exist_ok=True)

        self.dataset_dir = os.path.join("./datasets", f"Datasets_{self.config['transformation']}")

        # prepare loaders
        print("📦 Preparing datasets and transforms...")
        self.train_loader, self.valid_loader, self.test_loader, self.full_train_loader = self.prepare_loaders()

        # # run sanity checks on full-volume loader (not patch loader)
        # print("🔎 Running dataset sanity checks on full volumes (not patches)...")
        # self.print_dataset_stats(self.full_train_loader, name="train_full", n=30)
        # self.print_dataset_stats(self.valid_loader, name="valid", n=30)
        # self.print_dataset_stats(self.test_loader, name="test", n=30)

    def print_dataset_stats(self, loader, name="train", n=20):
        tot_pos = 0.0
        tot_vox = 0
        samples = 0
        unique_vals = set()
        for i, batch in enumerate(loader):
            seg = batch["seg"][0, 0]  # [D,H,W]
            seg_cpu = seg.cpu()
            tot_pos += float(seg_cpu.sum().item())
            tot_vox += seg_cpu.numel()
            samples += 1
            vals = torch.unique(seg_cpu)
            for v in vals.tolist():
                unique_vals.add(v)
            if i + 1 >= n:
                break
        frac = tot_pos / (tot_vox + 1e-12)
        print(f"  [{name}] checked {samples} samples. total pos voxels: {tot_pos:.1f}, total voxels: {tot_vox}, pos fraction: {frac:.6g}")
        print(f"  [{name}] example mask unique values (approx): {sorted(list(unique_vals))[:10]}")
        if frac == 0:
            print(f"  ⚠️ [{name}] No positives found in the inspected samples.")
        if any(v not in (0.0, 1.0) for v in unique_vals):
            print(f"  ⚠️ [{name}] Found non-binary mask values (not only 0/1). Use nearest interpolation for segs.")

        return frac

    def prepare_loaders(self):
        pixdim = (1, 1, 1)
        a_min, a_max = -1000, 700
        patch_size = (96, 96, 96)

        def get_files(split):
            ct = sorted(glob(os.path.join(self.dataset_dir, split, "ct", "*.nii.gz")))
            seg = sorted(glob(os.path.join(self.dataset_dir, split, "segment", "*.nii.gz")))
            if len(ct) != len(seg):
                raise RuntimeError(f"Mismatch CT/SEG for {split}: {len(ct)} vs {len(seg)}")
            return [{"vol": c, "seg": s} for c, s in zip(ct, seg)]


       # full-volume transforms for validation/test + sanity checks
        base_transforms = Compose([
            LoadImaged(keys=["vol", "seg"]),
            EnsureChannelFirstd(keys=["vol", "seg"]),
            # Spacingd: interpolation mode for vol then seg (image->bilinear, seg->nearest)
            Spacingd(keys=["vol", "seg"], pixdim=pixdim, mode=("bilinear", "nearest")),
            Orientationd(keys=["vol", "seg"], axcodes="RAS", labels=None),
            ScaleIntensityRanged(keys=["vol"], a_min=a_min, a_max=a_max, b_min=0.0, b_max=1.0, clip=True),
            CropForegroundd(keys=["vol", "seg"], source_key="vol"),
            # pad full volumes up to patch_size (constant padding)
            ResizeWithPadOrCropd(keys=["vol", "seg"], spatial_size=patch_size, mode="constant"),
            ToTensord(keys=["vol", "seg"]),
        ])

        # train transforms: pad/crop to >= patch_size, then sample patches with RandCropByPosNegLabeld
        train_transforms = Compose([
            LoadImaged(keys=["vol", "seg"]),
            EnsureChannelFirstd(keys=["vol", "seg"]),
            Spacingd(keys=["vol", "seg"], pixdim=pixdim, mode=("bilinear", "nearest")),
            Orientationd(keys=["vol", "seg"], axcodes="RAS", labels=None),
            ScaleIntensityRanged(keys=["vol"], a_min=a_min, a_max=a_max, b_min=0.0, b_max=1.0, clip=True),
            CropForegroundd(keys=["vol", "seg"], source_key="vol"),
            # Resize/pad so volume is at least patch_size (constant padding)
            ResizeWithPadOrCropd(keys=["vol", "seg"], spatial_size=patch_size, mode="constant"),

            # Balanced sampling by positive labels (num_samples=1; batch_size controls effective patches per step)
            RandCropByPosNegLabeld(
                keys=["vol", "seg"],
                label_key="seg",
                spatial_size=patch_size,
                pos=1,
                neg=1,
                num_samples=1,
                image_key="vol",
                image_threshold=0,
            ),

            # augmentations (image: bilinear / seg: nearest is handled above on Spacingd)
            RandFlipd(keys=["vol", "seg"], prob=0.5, spatial_axis=0),
            RandFlipd(keys=["vol", "seg"], prob=0.5, spatial_axis=1),
            RandAffined(keys=["vol", "seg"], prob=0.3,
                        rotate_range=(0.1, 0.1, 0.1),
                        scale_range=(0.1, 0.1, 0.1),
                        mode=("bilinear", "nearest")),  # interpolation for images/segs
            RandGaussianNoised(keys=["vol"], prob=0.2, mean=0.0, std=0.1),
            RandScaleIntensityd(keys=["vol"], factors=0.1, prob=0.5),

            ToTensord(keys=["vol", "seg"]),
            EnsureTyped(keys=["vol", "seg"]),
        ])

        # Create datasets and loaders
        train_ds = Dataset(get_files("train"), train_transforms)
        valid_ds = Dataset(get_files("valid"), base_transforms)
        test_ds  = Dataset(get_files("test"), base_transforms)

        # Optionally you can use CacheDataset for speed (uncomment & tune)
        # from monai.data import CacheDataset
        # train_ds = CacheDataset(data=get_files("train"), transform=train_transforms, cache_rate=0.5, num_workers=4)

        train_loader = DataLoader(train_ds, batch_size=self.config['batch_size'], shuffle=True, num_workers=0, collate_fn=pad_list_data_collate)
        valid_loader = DataLoader(valid_ds, batch_size=self.config['batch_size'], shuffle=False, num_workers=0, collate_fn=pad_list_data_collate)
        test_loader  = DataLoader(test_ds,  batch_size=1, shuffle=False, num_workers=0, collate_fn=pad_list_data_collate)

        # full volume train loader for sanity checks (no patching)
        full_train_ds = Dataset(get_files("train"), base_transforms)
        full_train_loader = DataLoader(full_train_ds, batch_size=1, shuffle=False, num_workers=0, collate_fn=pad_list_data_collate)

        print(f"✅ Dataset sizes — Train: {len(train_ds)}, Valid: {len(valid_ds)}, Test: {len(test_ds)}")

        return train_loader, valid_loader, test_loader, full_train_loader

    def train(self):
        trainer = UnetTrain(
            model_file=self.model_file,
            loss_result_path=self.loss_result_file,
            lr=self.config['learning_rate'],
            num_epochs=self.config['num_epochs'],
            device=self.device
        )
        trainer.execute(self.train_loader, self.valid_loader)

    def test(self):
        # Create model and load checkpoint for testing
        model = UNet(
            spatial_dims=3,
            in_channels=1,
            out_channels=1,
            channels=(16, 32, 64, 128, 256),
            strides=(2, 2, 2, 2),
            num_res_units=2,
            norm=Norm.BATCH
        ).to(self.device)

        if os.path.exists(self.model_file):
            checkpoint = torch.load(self.model_file, map_location=self.device)
            if checkpoint.get('model_state_dict') is not None:
                model.load_state_dict(checkpoint['model_state_dict'])
            else:
                model.load_state_dict(checkpoint)
        else:
            raise FileNotFoundError(f"Model file not found: {self.model_file}")

        tester = None  # your UnetTest implementation can be used here
        # tester = UnetTest(self.test_result_path, self.test_metrics_file, self.device)
        # tester.test(model, self.test_loader)

    def run(self):
        self.train()
        self.test()


# ---------- Run ----------
def main():
    import multiprocessing as mp
    mp.set_start_method('spawn', force=True)

    config = {
        'target_dir': "/content/drive/MyDrive/PhDwork/Segmentation",
        'output_folder_name': "Results_Nifti_MONAI2",
        'transformation': "OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train",
        'batch_size': 2,   # effective patches per step = num_samples * batch_size (we set num_samples=1)
        'num_epochs': 100,
        'learning_rate': 1e-4,
    }

    pipeline = UnetPipeline(config)
    pipeline.run()   # uncomment to actually train/test

if __name__ == "__main__":
    main()

📁 Current Directory: /content/drive/MyDrive/PhDwork/Segmentation
📦 Preparing datasets and transforms...
✅ Dataset sizes — Train: 340, Valid: 43, Test: 38
📊 Auto-computed pos_weight = 534.107 (pos voxels=165338, neg voxels=88308262)
Epoch 001 | Time: 69m 26s | Train: 2.789326 | Val: 3.488296
✅ Validation loss improved. Saving model...
Epoch 002 | Time: 66m 48s | Train: 2.702567 | Val: 3.285268
✅ Validation loss improved. Saving model...
Epoch 003 | Time: 67m 32s | Train: 2.620983 | Val: 3.209697
✅ Validation loss improved. Saving model...
Epoch 004 | Time: 67m 13s | Train: 2.600965 | Val: 3.113213
✅ Validation loss improved. Saving model...
Epoch 005 | Time: 67m 34s | Train: 2.558125 | Val: 3.082911
✅ Validation loss improved. Saving model...
Epoch 006 | Time: 68m 11s | Train: 2.547104 | Val: 3.094644
⏳ EarlyStopping counter: 1 out of 10


#(8) Mask Generation

In [ ]:
import os
import torch
import nibabel as nib
import numpy as np
from pathlib import Path
from monai.networks.nets import UNet
from monai.transforms import (
    Compose,
    Resized,
    CopyItemsd,
    Invertd,
    LoadImaged,
    EnsureChannelFirstd,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
    EnsureTyped,
    SaveImaged,
    ToTensord,
)
from monai.data import Dataset, DataLoader, decollate_batch
from monai.inferers import sliding_window_inference
from monai.utils import set_determinism
from monai.networks.layers import Norm
# from monai.transforms.utils import SaveTransform



class UNetInferencePipeline:
    def __init__(self, model_path, input_ct_dir, input_seg_dir, output_dir, device="cuda:0"):
        self.device = device if torch.cuda.is_available() else "cpu"
        self.input_ct_dir = input_ct_dir
        self.input_seg_dir = input_seg_dir
        self.output_dir = output_dir
        self.ct_out_dir = os.path.join(output_dir, "ct")
        self.seg_out_dir = os.path.join(output_dir, "segment")
        os.makedirs(self.ct_out_dir, exist_ok=True)
        os.makedirs(self.seg_out_dir, exist_ok=True)
        self.model_path = model_path
        self.model = self._load_model()
        set_determinism(seed=42)
        self.forward_transforms = self._get_forward_transforms()
        self.inverse_transforms = None
        self.dataloader = self._prepare_dataloader()

    def _load_model(self):
        if not os.path.exists(self.model_path):
            raise FileNotFoundError(f"Model file not found at: {self.model_path}")

        model = UNet(
            spatial_dims=3,
            in_channels=1,
            out_channels=1,
            channels=(16, 32, 64, 128, 256),
            strides=(2, 2, 2, 2),
            num_res_units=2,
            norm=Norm.BATCH
        ).to(self.device)

        state_dict = torch.load(self.model_path, map_location=self.device)
        model.load_state_dict(state_dict.get('model_state_dict', state_dict))

        print(f"✅ Model loaded successfully from {self.model_path}")
        return model



    def _get_forward_transforms(self):
        return Compose([
            LoadImaged(keys=["vol"]),
            EnsureChannelFirstd(keys=["vol"]),
            CopyItemsd(keys=["vol"], names=["vol_meta_dict"]),
            Spacingd(keys=["vol"], pixdim=(1.0, 1.0, 1.0), mode="bilinear"),
            Orientationd(keys=["vol"], axcodes="RAS"),
            ScaleIntensityRanged(keys=["vol"], a_min=-1000, a_max=700, b_min=0.0, b_max=1.0, clip=True),
            CropForegroundd(keys=["vol"], source_key="vol"),
            Resized(keys=["vol"], spatial_size=(96, 96, 96)),
            EnsureTyped(keys=["vol"]),
        ])

    def _get_inverse_transforms(self):
        return Compose([
            Invertd(
                keys=["seg"],
                transform=self.forward_transforms,
                orig_keys=["vol"],
                meta_keys=["vol_meta_dict"],
                nearest_interp=True,
                to_tensor=False,
            ),
            EnsureTyped(keys=["seg"])
        ])

    def _prepare_dataloader(self):
        data = []
        for f in os.listdir(self.input_ct_dir):
            if f.endswith(('.nii', '.nii.gz')):
                ct_path = os.path.join(self.input_ct_dir, f)
                data.append({"vol": ct_path})
        print(f"🔍 Found {len(data)} NIfTI files for inference.")
        return DataLoader(Dataset(data=data, transform=self.forward_transforms), batch_size=1, num_workers=0)

    def infer(self):
        self.model.eval()
        with torch.no_grad():
            for i, batch in enumerate(self.dataloader):
                batch = decollate_batch(batch)[0]
                vol_meta = batch["vol_meta_dict"]
                ct = batch["vol"]

                if ct.dim() == 4:
                    ct = ct.unsqueeze(0)
                ct = ct.to(self.device)

                filename = os.path.basename(vol_meta.meta["filename_or_obj"])
                orig_vol = nib.load(vol_meta.meta["filename_or_obj"]).get_fdata()
                print(f"🔍 Inference on [{i+1}] {filename} | shape = {ct.shape}")
                print(f"🔍 Original volume shape = {orig_vol.shape}")
                pred = self.model(ct)
                pred = (torch.sigmoid(pred) > 0.5).float()

                print(f"✅ Predicted mask shape: {pred.shape}")

                batch["seg"] = pred.cpu().squeeze(0)
                print(f"✅ Batch shape: {batch['seg'].shape}")

                if self.inverse_transforms is None:
                    self.inverse_transforms = self._get_inverse_transforms()

                inverted = self.inverse_transforms(batch)
                inv_seg = inverted["seg"].squeeze(0).numpy()
                inv_seg = (inv_seg > 0.5).astype(np.uint8)
                print(f"✅ Inverted mask shape: {inv_seg.shape}")

                self._save_nifti(inv_seg, vol_meta, self.seg_out_dir, filename, is_segmentation=True)


    def _save_nifti(self, array, meta_tensor, out_dir, filename, is_segmentation=False):
        os.makedirs(out_dir, exist_ok=True)
        affine = meta_tensor.meta.get("original_affine", meta_tensor.meta.get("affine", np.eye(4)))
        dtype = np.uint8 if is_segmentation else np.float32
        nib_img = nib.Nifti1Image(array.astype(dtype), affine)
        nib.save(nib_img, os.path.join(out_dir, filename))
        print(f"✅ Saved: {os.path.join(out_dir, filename)}")


if __name__ == "__main__":
    ROOT_DIR = "/content/drive/MyDrive/PhDwork/Segmentation"
    MODEL_PATH = os.path.join(ROOT_DIR, "results", "Results_MONAI_Augmented", "model.pth")
    INPUT_CT_FOLDER = os.path.join(ROOT_DIR, "datasets", "Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train", "Lung3", "ct")
    INPUT_SEG_FOLDER = os.path.join(ROOT_DIR, "datasets", "Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train", "Lung3", "segment")
    OUTPUT_FOLDER = os.path.join(ROOT_DIR, "datasets", "Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train", "Lung3_Predicted")

    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    os.chdir(ROOT_DIR)

    try:
        pipeline = UNetInferencePipeline(MODEL_PATH, INPUT_CT_FOLDER, INPUT_SEG_FOLDER, OUTPUT_FOLDER)
        pipeline.infer()
        print("🎉 Inference completed successfully for all patients!")
    except FileNotFoundError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


✅ Model loaded successfully from /content/drive/MyDrive/PhDwork/Segmentation/results/Results_MONAI_Augmented/model.pth
🔍 Found 89 NIfTI files for inference.
🔍 Inference on [1] LUNG3-01.nii.gz | shape = torch.Size([1, 1, 96, 96, 96])
🔍 Original volume shape = (59, 512, 512)
✅ Predicted mask shape: torch.Size([1, 1, 96, 96, 96])
✅ Batch shape: torch.Size([1, 96, 96, 96])
✅ Inverted mask shape: (59, 512, 512)
✅ Saved: /content/drive/MyDrive/PhDwork/Segmentation/datasets/Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train/Lung3_Predicted/segment/LUNG3-01.nii.gz
🔍 Inference on [2] LUNG3-02.nii.gz | shape = torch.Size([1, 1, 96, 96, 96])
🔍 Original volume shape = (57, 512, 512)
✅ Predicted mask shape: torch.Size([1, 1, 96, 96, 96])
✅ Batch shape: torch.Size([1, 96, 96, 96])
✅ Inverted mask shape: (57, 512, 512)
✅ Saved: /content/drive/MyDrive/PhDwork/Segmentation/datasets/Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train/Lung3_Predicted/segment/LUNG3-02.nii.gz
🔍 Inference on [3] 

In [ ]:
if __name__ == "__main__":
    ROOT_DIR = "/content/drive/MyDrive/PhDwork/Segmentation"
    MODEL_PATH = os.path.join(ROOT_DIR, "results", "Results_MONAI_Augmented", "model.pth")
    INPUT_CT_FOLDER = os.path.join(ROOT_DIR, "datasets", "Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train", "test", "ct")
    INPUT_SEG_FOLDER = os.path.join(ROOT_DIR, "datasets", "Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train", "test", "segment")
    OUTPUT_FOLDER = os.path.join(ROOT_DIR, "datasets", "Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train", "test_Predicted")

    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    os.chdir(ROOT_DIR)

    try:
        pipeline = UNetInferencePipeline(MODEL_PATH, INPUT_CT_FOLDER, INPUT_SEG_FOLDER, OUTPUT_FOLDER)
        pipeline.infer()
        print("🎉 Inference completed successfully for all patients!")
    except FileNotFoundError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

✅ Model loaded successfully from /content/drive/MyDrive/PhDwork/Segmentation/results/Results_MONAI_Augmented/model.pth
🔍 Found 38 NIfTI files for inference.
🔍 Inference on [1] LUNG1-001.nii.gz | shape = torch.Size([1, 1, 96, 96, 96])
🔍 Original volume shape = (134, 512, 512)
✅ Predicted mask shape: torch.Size([1, 1, 96, 96, 96])
✅ Batch shape: torch.Size([1, 96, 96, 96])
✅ Inverted mask shape: (134, 512, 512)
✅ Saved: /content/drive/MyDrive/PhDwork/Segmentation/datasets/Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train/test_Predicted/segment/LUNG1-001.nii.gz
🔍 Inference on [2] LUNG1-025.nii.gz | shape = torch.Size([1, 1, 96, 96, 96])
🔍 Original volume shape = (106, 512, 512)
✅ Predicted mask shape: torch.Size([1, 1, 96, 96, 96])
✅ Batch shape: torch.Size([1, 96, 96, 96])
✅ Inverted mask shape: (106, 512, 512)
✅ Saved: /content/drive/MyDrive/PhDwork/Segmentation/datasets/Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train/test_Predicted/segment/LUNG1-025.nii.gz
🔍 Inference o

In [ ]:
if __name__ == "__main__":
    ROOT_DIR = "/content/drive/MyDrive/PhDwork/Segmentation"
    MODEL_PATH = os.path.join(ROOT_DIR, "results", "Results_MONAI_Augmented", "model.pth")
    INPUT_CT_FOLDER = os.path.join(ROOT_DIR, "datasets", "Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train", "valid", "ct")
    INPUT_SEG_FOLDER = os.path.join(ROOT_DIR, "datasets", "Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train", "valid", "segment")
    OUTPUT_FOLDER = os.path.join(ROOT_DIR, "datasets", "Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train", "valid_Predicted")

    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    os.chdir(ROOT_DIR)

    try:
        pipeline = UNetInferencePipeline(MODEL_PATH, INPUT_CT_FOLDER, INPUT_SEG_FOLDER, OUTPUT_FOLDER)
        pipeline.infer()
        print("🎉 Inference completed successfully for all patients!")
    except FileNotFoundError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

✅ Model loaded successfully from /content/drive/MyDrive/PhDwork/Segmentation/results/Results_MONAI_Augmented/model.pth
🔍 Found 43 NIfTI files for inference.
🔍 Inference on [1] LUNG1-010.nii.gz | shape = torch.Size([1, 1, 96, 96, 96])
🔍 Original volume shape = (91, 512, 512)
✅ Predicted mask shape: torch.Size([1, 1, 96, 96, 96])
✅ Batch shape: torch.Size([1, 96, 96, 96])
✅ Inverted mask shape: (91, 512, 512)
✅ Saved: /content/drive/MyDrive/PhDwork/Segmentation/datasets/Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train/valid_Predicted/segment/LUNG1-010.nii.gz
🔍 Inference on [2] LUNG1-031.nii.gz | shape = torch.Size([1, 1, 96, 96, 96])
🔍 Original volume shape = (153, 512, 512)
✅ Predicted mask shape: torch.Size([1, 1, 96, 96, 96])
✅ Batch shape: torch.Size([1, 96, 96, 96])
✅ Inverted mask shape: (153, 512, 512)
✅ Saved: /content/drive/MyDrive/PhDwork/Segmentation/datasets/Datasets_OriginalCT_Nifti_Empty_NonEmpty_slices_In_Train/valid_Predicted/segment/LUNG1-031.nii.gz
🔍 Inference o

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from typing import List
import os
import csv


class LossPlotter:
    def __init__(self, csv_path: str):
        self.csv_path = Path(csv_path)
        self.data = self._load_data()

    def _load_data(self):
        if not self.csv_path.exists():
            raise FileNotFoundError(f"CSV file not found: {self.csv_path}")
        df = pd.read_csv(self.csv_path, index_col=0)  # Read row labels as index
        return df  # Make rows into columns

    def plot(self, title: str = "Training and Validation Loss", save_path= None):
        plt.figure(figsize=(8, 5))
        plt.plot(self.data.index, self.data['Train Loss'], label='Train Loss', color='blue')
        plt.plot(self.data.index, self.data['Valid Loss'], label='Valid Loss', color='orange')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title(title)
        plt.legend()
        plt.grid(True)
        plt.tight_layout()

        if save_path:
            save_path = Path(save_path)
            save_path.parent.mkdir(parents=True, exist_ok=True)
            plt.savefig(save_path, format='pdf')
            print(f"[INFO] Loss plot saved to {save_path}")
        else:
            plt.show()

        plt.close()

if __name__ == "__main__":
    target_dir = "/content/drive/MyDrive/PhDwork/Segmentation"
    os.chdir(target_dir)
    loss_result_file = os.path.join(".","results",f"Results_PreProcessedCT_Fifty_Fifty_DiceLoss_And_Strong_Augmentation","train_and_valid_loss_results.csv")
    plotter = LossPlotter(loss_result_file)
    plotter.plot()


In [ ]:
import h5py
os.chdir("/content/drive/MyDrive/PhDwork/Segmentation")
print(f"📁 Current Directory: {os.getcwd()}")
with h5py.File('./datasets/Datasets_PreprocessedCT_clipping_uniformSpacing_With_Empty_NonEmpty_slices_In_Train/train_dataset.hdf5', 'r') as f:
    print(list(f.keys()))